In [1]:
%matplotlib inline

proj_path = "/Users/andrew/Desktop/projects/hidden_singles/"

In [2]:
import sys
sys.path.append(proj_path + 'python')

import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from hiddensingles.misc import utils
from PIL import Image, ImageDraw, ImageFont

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 300)

In [10]:
# Colors used in the experiment
c_bg_green = (1, 255, 112)
c_bg_blue = (127, 219, 255)
c_bg_purple = (218, 112, 214)
c_bg_red = (255, 69, 0)
c_bg_orange = (255, 165, 0)
c_digit_blue = (0, 0, 255)

In [6]:
def img_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

def img_add_title(img, title, bg_color=(255, 255, 255)):
    img_title = Image.new('RGB', (img.width, int(.12 * img.height)), color=bg_color)
    draw = ImageDraw.Draw(img_title)
    fnt = ImageFont.truetype('/Library/Fonts/Arial.ttf', int(.65 * img_title.height))
    text_w, text_h = draw.textsize(title, font=fnt)
    x = int((img_title.width - text_w)) / 2
    y = int((img_title.height - text_h)) / 2
    draw.text((x, y), title, font=fnt, fill='black')
    return img_concat_v(img_title, img)

In [47]:
def render_sudoku(df_puzzle, size=400):
    """
    df_puzzle must contain the following columns:
        key, type, row, column, number, bg_color, color
    0s are not added to the figure
    """
    line_house_width = size // 100
    line_cell_width = size // 400
    line_exterior_width = line_house_width * 2
    
    df_puzzle['bg_color'] = [c_bg_blue if t in ('inhouse', 'empty') else None for t in df_puzzle.type]
    df_puzzle['bg_color'] = [c_bg_green if k == 'goal' else c for k, c in zip(df_puzzle.key, df_puzzle.bg_color)]
    df_puzzle['color'] = [(0, 0, 0) if k == 'goal' else c_digit_blue for k in df_puzzle.key]
    
    img = Image.new('RGB', (size, size), color=(255, 255, 255))
    img_max = size 
    house_width = int(img_max / 3)
    cell_width = int(house_width / 3)

    fnt = ImageFont.truetype('/Library/Fonts/Arial.ttf', int(.1 * size))
    draw = ImageDraw.Draw(img)

    # Add highlighted cells
    for row, column, color in zip(df_puzzle.row, df_puzzle.column, df_puzzle.bg_color):
        if color is not None:
            draw.rectangle(((column-1) * cell_width,
                            (row-1) * cell_width,
                            column * cell_width,
                            row * cell_width), fill=color)

    # Create exterior borders
    draw.line((0, 0) + (0, img_max), fill=0, width=line_exterior_width)
    draw.line((0, 0) + (img_max, 0), fill=0, width=line_exterior_width)
    draw.line((img_max, 0) + (img_max, img_max), fill=0, width=line_exterior_width)
    draw.line((0, img_max) + (img_max, img_max), fill=0, width=line_exterior_width)

    # Create house borders
    draw.line((0, house_width) + (img_max, house_width), fill=0, width=line_house_width)
    draw.line((0, 2 * house_width) + (img_max, 2 * house_width), fill=0, width=line_house_width)
    draw.line((house_width, 0) + (house_width, img_max), fill=0, width=line_house_width)
    draw.line((2 * house_width, 0) + (2 * house_width, img_max), fill=0, width=line_house_width)

    # Create cell borders
    for i in range(9):
        draw.line((0, i * cell_width) + (img_max, i * cell_width), fill=0, width=line_cell_width)
        draw.line((i * cell_width, 0) + (i * cell_width, img_max), fill=0, width=line_cell_width)

    # Add numbers
    for row, column, number, color in zip(df_puzzle.row, df_puzzle.column, df_puzzle.number, df_puzzle.color):
        if number > 0:
            draw.text((((column-1) + .27) * cell_width,
                       ((row-1) + .025) * cell_width), str(number),
                        font=fnt, fill=color)

    return img

# Questionnaire V2

In [52]:
df_qratings2 = pd.read_csv("../../data/qratings/v2_responses.tsv", sep='\t')[['qid2', 'sid_hash']]
df_house_types = pd.read_table("../../data/processed/house_types.tsv")
df_puzzles = pd.read_table("../../data/processed/puzzles.tsv")
df_puzzles = df_puzzles.merge(df_house_types)
df_puzzles = df_puzzles[df_puzzles.phase == 'questionnaire']
df_puzzles = df_qratings2.merge(df_puzzles)

In [53]:
images = []
for qid2, df_puzzle in df_puzzles.groupby('qid2'):
    img = render_sudoku(df_puzzle, 900)
    img = img_add_title(img, f"Subject {qid2}")
    images.append(img)
    
images[0].save(fp="../../figures/q_ratings/puzzles_v2.pdf",
               append_images=images[1:],
               save_all=True,
               dpi=(600, 600)) 